In [ ]:
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using LinearAlgebra
using HybridRobotDynamics
using SiLQR

In [ ]:
"""
Cartpole Model
"""

function get_cartpole_model(
    mc::Float64,
    mp::Float64,
    l::Float64,
    g::Float64 = 9.81
)::HybridSystem
    # States: cart position, pole angle, cart velocity, pole angular velocity
    function cartpole_flow(x::Vector, u::Vector)::Vector
        θ = x[2]
        θ̇ = x[4]

        M = [mc+mp mp*l*cos(θ); mp*l*cos(θ) mp*l^2]
        C = [0.0 -mp*l*θ̇*sin(θ); zeros(1, 2)]
        τ = [0, -mp*g*l*sin(θ)]
        B = [1.0, 0.0]

        q̇ = x[3:4]
        q̈ = M \ (τ + B*u[1] - C*q̇)
        return [q̇; q̈]
    end

    # Return HybridSystem
    nx = 4
    nu = 1
    mI = HybridMode(cartpole_flow)
    modes = Dict(:nominal => mI)
    transitions = Dict(Tuple{Symbol, Transition}[])
    return HybridSystem(nx, nu, transitions, modes)
end

nothing

In [ ]:
"""
Solver Setup
"""

mc = 1.2
mp = 0.16
l = 0.55
system = get_cartpole_model(mc, mp, l)

# Stage and terminal costs
Q = 1e-3 * diagm([0.1, 1.0, 1.0, 1.0])
R = 1e-6* I(system.nu)
Qf = 1e3 * Q
stage(x, u) = x'*Q*x + u'*R*u
terminal(x) = x'*Qf*x

# RK4 integrator
rk4 = ExplicitIntegrator(:rk4)

# Problem parameters
N = 50
Δt = 0.1
params = SiLQR.Parameters(system, stage, terminal, rk4, N, Δt)

# Reference trajectory
xref = [0.0, pi, 0.0, 0.0]
uref = zeros(system.nu)
params.xrefs = [xref for k = 1:N]
params.urefs = [uref for k = 1:(N-1)]

# Initial conditions
params.x0 = 1e-3 * ones(system.nx)
params.mI = :nominal

nothing

In [ ]:
"""
Solve using SiLQR
"""

# Solve
sol = SiLQR.Solution(params)
cache = SiLQR.Cache(params)
@time SiLQR.solve!(sol, cache, params)

# Visualize states
xs = reduce(vcat, sol.xs)
plot_2d_states(N, system.nx, (1,2), xs; xlim=(-1,4), ylim=(-1,4))

nothing